In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap

In [ ]:
# Import Data
# Maxar
maxar = ee.Image("users/gfiske/FrostCraters/maxar_imagery_v2");

# Planet
planet = ee.ImageCollection('projects/abruptthawmapping/assets/yg_polygons_composites')

# Polygons
polygons = ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_for_Yili_Apr_2022').filterMetadata('code_id', 'equals', 1)

In [ ]:
# Get Sentinel-2 Mosaic to Compare, as well
MAX_CLOUD_PROBABILITY = 50
geometry = ee.Geometry({
    'type': 'Polygon',
    'coordinates': [[
        [66, 66],
        [66, 73.5],
        [85, 73.5],
        [85, 66],
        [66, 66]
        ]]
})

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2017-06-15', '2021-08-31')
    .filter(ee.Filter.dayOfYear(135, 243))
);

s2_clouds = (
    ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    .filterBounds(geometry)
    .filterDate('2017-06-15', '2021-08-31')
    .filter(ee.Filter.dayOfYear(135, 243))
);

def s2_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2_with_clouds = ee.Join.saveFirst('cloud_mask').apply(    
  primary=s2,
  secondary=s2_clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

s2_masked = ee.ImageCollection(s2_with_clouds).map(s2_maskcloud)

s2_mosaic = (
    s2_masked
    .select(['B2', 'B3', 'B4', 'B8'])
    .median()
    .clip(geometry));

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

planet = planet.map(mask_0)

In [ ]:
# Set Properties to allow filtering
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;


planet = planet.map(setID);

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet)

In [ ]:
# Separate out Years
planet_2018 = planet.filter(ee.Filter.stringEndsWith('ID', ee.String('2018')))
planet_2019 = planet.filter(ee.Filter.stringEndsWith('ID', ee.String('2019')))
planet_2020 = planet.filter(ee.Filter.stringEndsWith('ID', ee.String('2020')))
planet_2021 = planet.filter(ee.Filter.stringEndsWith('ID', ee.String('2021')))

In [ ]:
# Set visualization parameters
vis_params = {
    'min': [350, 325, 200],'max': [1200, 1175, 1000],
    'bands': ['b3', 'b2', 'b1'],
    'gamma': 1
}

In [ ]:
# Add layers
# Add Maxar to all maps
Map.addLayer(maxar, {'min':8, 'max':192}, 'Maxar')

# Add Sentinel-2 composite
Map.addLayer(s2_mosaic, {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1200}, 'S2 All Years')

# add Planet composites
Map.addLayer(planet_2018, vis_params, 'Planet 2018')
Map.addLayer(planet_2019, vis_params, 'Planet 2019')
Map.addLayer(planet_2020, vis_params, 'Planet 2020')
Map.addLayer(planet_2021, vis_params, 'Planet 2021')

# add RTS Polygons
Map.addLayer(polygons, {'color': 'red'}, 'RTS')

In [ ]:
# Display the map
Map